In [1]:
import pandas as pd
import re
import numpy as np


In [2]:
df = pd.read_csv("babyfoot-dataset-csv-4.csv")

C:\Users\charl\AppData\Local\Temp\ipykernel_3276\4035095961.py:1: DtypeWarning: Columns (10,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("babyfoot-dataset-csv-4.csv")


## Fix the dataset

### fonctions

In [3]:


def parse_scores(value):
    """Détecte les cas 'X - Y' ou 'X–Y' et retourne (X, Y)"""
    if pd.isna(value):
        return None, None
    s = str(value).strip()
    # Normaliser : remplacer tirets exotiques et espaces
    s = re.sub(r"[–—−]", "-", s.replace(" ", ""))
    # Si on trouve le format "nombre-nombre"
    match = re.match(r"^(\d+)-(\d+)$", s)
    if match:
        red = int(match.group(1))
        blue = int(match.group(2))
        return red, blue
    return None, None

def to_int_or_none(v):
    """Convertit proprement une valeur en int ou None"""
    if pd.isna(v):
        return None
    m = re.search(r'\d+', str(v))
    return int(m.group()) if m else None

def compute_winner(row):
    r, b = row["final_score_red"], row["final_score_blue"]
    if pd.isna(r) or pd.isna(b):
        return None
    if r > b:
        return "red"
    elif b > r:
        return "blue"
    else:
        return "tie"

def delete_duplicates(df):
    df_clean = df.drop_duplicates()
    return df_clean

def convert_time_to_seconds(value):
    if pd.isna(value):
        return None

    # 1. Nettoyer le texte
    s = str(value).lower().strip()
    s = s.replace('minutes', '').replace('minute', '').replace('mins', '').replace('min', '')
    s = s.replace('s', '').replace(':', '.')
    s = re.sub(r'\s+', '', s)  # supprime espaces

    # 2. Si c’est juste un nombre → secondes directes
    if s.isdigit():
        return int(s)

    # 3. Si format "m.ss" ou "m.sss"
    if re.match(r'^\d+(\.\d+)?$', s):
        parts = s.split('.')
        minutes = int(parts[0])
        secondes = int(parts[1]) if len(parts) > 1 else 0
        return minutes * 60 + secondes
    return None

def convert_is_substitute(x):
    """Convertit 1.0 -> True, 0.0 -> False, sinon NaN."""
    if x == 1.0:
        return True
    elif x == 0.0:
        return False
    else:
        return np.nan


### Code

drop minutes, enlever les deux points > point, faire *60 des chiffre avant le point te add avant et après le point  

In [4]:
#supprimer les doublons 

df = delete_duplicates(df)

# Appliquer la conversion en secondes
df["game_duration"] = df["game_duration"].apply(convert_time_to_seconds)
df["possession_time"] = df["possession_time"].apply(convert_time_to_seconds)



# Correction des scores stockés que dans la première colonne
new_red, new_blue = [], []

for red_val, blue_val in zip(df["final_score_red"], df.get("final_score_blue", [None]*len(df))):
    r, b = parse_scores(red_val)
    if r is not None and b is not None:        # Cas '0 - 10'
        new_red.append(r)
        new_blue.append(b)
    else:                                     # Cas normal
        new_red.append(to_int_or_none(red_val))
        new_blue.append(to_int_or_none(blue_val))

df["final_score_red"] = new_red
df["final_score_blue"] = new_blue


df["winner"] = df.apply(compute_winner, axis=1)




### Nettoyage de 'game_date'


In [5]:
df["game_date"] = df["game_date"].astype(str).str.strip().replace(
    {"": np.nan, "nan": np.nan, "NaN": np.nan, "NULL": np.nan, "None": np.nan}
)

# Conversion en datetime
df["game_date"] = pd.to_datetime(df["game_date"], errors="coerce", dayfirst=True)

# Garde uniquement les dates plausibles
today = pd.Timestamp.today().normalize()
mask_plausible = df["game_date"].between("2010-01-01", today + pd.Timedelta(days=1)) | df["game_date"].isna()
df.loc[~mask_plausible, "game_date"] = pd.NaT

# 👉 Ici on reste en datetime64 pour pouvoir utiliser .dt
df["game_year"] = df["game_date"].dt.year.astype("Int64")
df["game_month"] = df["game_date"].dt.month.astype("Int64")
df["game_day"] = df["game_date"].dt.day.astype("Int64")
df["game_date"] = df["game_date"].dt.date


### Nettoyage de 'created_at'


In [6]:
df["created_at"] = pd.to_datetime(df["created_at"], errors="coerce")
df["created_at"] = df["created_at"].dt.strftime("%Y-%m-%d %H:%M:%S")


### Conversion numérique en entiers


In [7]:
cols_int = ["game_duration", "attendance_count", "possession_time", "player_age"]
for col in cols_int:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce").astype("Int64")

### Conversion booléenne de 'is_substitute'

In [8]:
df["is_substitute"] = df["is_substitute"].apply(convert_is_substitute)

## Save

In [9]:

df.to_csv("babyfoot_clean.csv", index=False)

In [ ]:
df

,game_id,game_date,location,table_id,table_condition,ball_type,music_playing,referee,game_duration,final_score_red,...,possession_time,mood,player_comment,team_color,is_substitute,notes,created_at,game_year,game_month,game_day
0,G015295,2023-06-02,Toulouse,T05,beer stains,NaN,Queen - We Will Rock You,Paul Kim,765,0,...,576,1.0,ref biased,red,True,injured,2025-10-02 10:41:54,2023,6,2
1,G023800,NaT,Toulouse,T07,worn,NaN,Indie playlist,NaN,537,10,...,<NA>,3.0,NaN,blue,True,NaN,2025-10-02 10:41:55,<NA>,<NA>,<NA>
2,G023577,NaT,Toulouse,T26,scratched,NaN,Queen - We Will Rock You,Lena Clement,1022,2,...,309,2.0,NaN,red,False,double booked,2025-10-02 10:41:55,<NA>,<NA>,<NA>
3,G020644,NaT,Toulouse,T21,worn,mini ball,EDM mix,Isabella Girard,318,6,...,<NA>,3.0,team spirit high,blue,NaN,double booked,2025-10-02 10:41:54,<NA>,<NA>,<NA>
4,G011677,NaT,Toulouse,T26,missing screw,trainer ball,Oldies 80s,yes,6,3,...,177,5.0,rage quit,red,True,NaN,2025-10-02 10:41:54,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100195,G007755,NaT,Toulouse,T02,needs cleaning,orange soft,NaN,no,407,0,...,<NA>,2.0,no comment,blue,NaN,NaN,2025-10-02 10:41:53,<NA>,<NA>,<NA>
100196,G011142,NaT,Toulouse,T06,out of alignment,classic,Indie playlist,Sam Petit,<NA>,7,...,<NA>,1.0,no comment,red,False,double booked,2025-10-02 10:41:54,<NA>,<NA>,<NA>
100197,G023955,NaT,Toulouse,T07,good,white hard,EDM mix,NaN,<NA>,6,...,485,3.0,NaN,red,False,NaN,2025-10-02 10:41:55,<NA>,<NA>,<NA>
100198,G018044,2023-09-01,Toulouse,T12,out of alignment,NaN,Radio 104.5,Jamie Blanc,777,1,...,548,5.0,ref biased,blue,NaN,late,2025-10-02 10:41:54,2023,9,1


: 